<br></br>

<div align="center">

<h1 align="center">
    Cuaderno para el trabajo de clasificación relacional
    <br></br>
    Entrenar modelos
</h1>

<h6 align="center">
    Antonio Macías Ferrera (antmacfer1@alum.us.es)
    <br></br>
    Delfín Santana Rubio (delsanrub@alum.us.es)
    <br></br>
    Universidad de Sevilla
</h6>

<br></br>

## **Control de Versiones**
    
| **Fecha**  | **Versión** | **Descripción**               |
| :--------- | :---------- | :---------------------------- |
| 27/05/2024 | v1r0        | Primera versión del cuaderno. |
| 31/05/2024 | v1r1        | Correcciones a algunos métodos e inclusión de otros.|
| 07/06/2024 | v1r2        | Correcciones de formato.      |


</div>

<br></br>

## **Índice de contenido**

1. [Introducción](#introducción)
2. [Instanciación del grafo](#creacion)
3. [Entrenamiento de los modelos](#entrenamiento)
    - [Árboles de decisión CART](#cart)
    - [Naive Bayes](#naive-bayes)
    - [KNN](#knn)
    - [Random Forest](#random-forest)
    - [Gradient Boosting](#gradient-boosting)

<br></br>

# <a name="introducción"></a> 1. **Introducción**

En este cuadernillo se encuentra todo el codigo en general que hemos utilizado para entrenar modelos.

A continuación se importan todas las librerías y métodos necesarios para la ejecución del código.

<br></br>

In [ ]:
# Pandas
import pandas as pd

# Numpy
import numpy as np

# Codificadores de sci-kit learn
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

# Modelos a entrenar (CART, Naive Bayes, Knn, Support Vector Machines)
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Evaluación del modelo
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score

# Discretizador para Naive Bayes
from sklearn.preprocessing import KBinsDiscretizer

# Normalizador para Knn
from sklearn.preprocessing import MinMaxScaler

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


# <a name="creacion"></a> 2. **Instanciación del grafo**

Aquí se cargan los datos del csv en un dataframe de Pandas, y se determinan cuales serán los atributos de entrenamiento (las métricas seleccionadas) y el atributo objetivo (tipos de grupos de Facebook).
<br></br>

In [50]:
values = pd.read_csv('dict.csv')
values.head()

,greedy_modularity_communities_id,label_propagation_communities_id,betweenness_centrality,degree_centrality,k_path_centrality,closeness_centrality,harmonic_centrality,square_clustering,clustering,target
0,5,0,0.000000,0.000045,0.000003,0.182482,4224.448882,0.000000,0.000000,tvshow
1,2,1,0.000153,0.001513,0.000100,0.257752,6230.390079,0.130612,0.481283,government
2,4,5,0.000003,0.000534,0.000035,0.189540,4419.497403,0.347089,0.651515,company
3,22,6,0.000052,0.000445,0.000029,0.219007,5147.241270,0.160836,0.511111,government
4,3,8,0.000125,0.002270,0.000149,0.230387,5493.355556,0.176843,0.420392,politician


In [51]:
atributos_continuos = ["degree_centrality", "k_path_centrality", "closeness_centrality", "harmonic_centrality", "clustering"]
atributos = values.loc[:, atributos_continuos]
atributos.head()

,degree_centrality,k_path_centrality,closeness_centrality,harmonic_centrality,clustering
0,0.000045,0.000003,0.182482,4224.448882,0.000000
1,0.001513,0.000100,0.257752,6230.390079,0.481283
2,0.000534,0.000035,0.189540,4419.497403,0.651515
3,0.000445,0.000029,0.219007,5147.241270,0.511111
4,0.002270,0.000149,0.230387,5493.355556,0.420392


In [52]:
objetivo = values['target']
objetivo.head()

0        tvshow
1    government
2       company
3    government
4    politician
Name: target, dtype: object

In [53]:
codificador_objetivo = LabelEncoder()
# El método fit_transform ajusta el codificador a los datos y, a continuación,
# codifica estos adecuadamente. En este caso no necesitamos mantener el
# atributo objetivo como una Series de Pandas.
objetivo = codificador_objetivo.fit_transform(objetivo)
print(f'Clases detectadas: {codificador_objetivo.classes_}')
objetivo

Clases detectadas: ['company' 'government' 'politician' 'tvshow']


array([3, 1, 0, ..., 1, 0, 3])

# <a name="entrenamiento"></a> 3. **Entrenamiento de los modelos**

<br></br>

## <a name="cart"></a> **Árboles de decisión CART**

In [54]:
# Arbol de decision

clasificador_CART = DecisionTreeClassifier(
    max_depth=12
)
clasificador_CART.fit(atributos, objetivo)

DecisionTreeClassifier(max_depth=12)

In [55]:
predicciones = clasificador_CART.predict(atributos)
confusion_matrix(objetivo, predicciones)
recall_score(objetivo, predicciones, average='micro')

0.6422340898976413

In [56]:
print("Parámetros usados: ")
clasificador_CART.get_params()

Parámetros usados: 


{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 12,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'random_state': None,
 'splitter': 'best'}

## <a name="naive-bayes"></a> **Naive Bayes**

In [57]:
discretizador = KBinsDiscretizer(
    n_bins=20,  # Cada atributo se discretiza en 4 intervalos
    encode='ordinal',  # Los intervalos se codifican numéricamente
    strategy='quantile'  # Cada intervalo contiene la misma cantidad de datos
)

# Como nos interesa conservar los atributos continuos originales, realizamos
# la discretización sobre una copia del DataFrame de atributos
atributos_discretizados = atributos.copy()
atributos_discretizados[atributos_continuos] = discretizador.fit_transform(
    atributos_discretizados[atributos_continuos]
)
atributos_discretizados.head()

/opt/homebrew/lib/python3.12/site-packages/sklearn/preprocessing/_discretization.py:322: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/opt/homebrew/lib/python3.12/site-packages/sklearn/preprocessing/_discretization.py:322: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 1 are removed. Consider decreasing the number of bins.
  warnings.warn(
/opt/homebrew/lib/python3.12/site-packages/sklearn/preprocessing/_discretization.py:322: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 4 are removed. Consider decreasing the number of bins.
  warnings.warn(


,degree_centrality,k_path_centrality,closeness_centrality,harmonic_centrality,clustering
0,0.0,0.0,4.0,4.0,0.0
1,13.0,13.0,19.0,19.0,9.0
2,9.0,9.0,5.0,5.0,11.0
3,8.0,8.0,13.0,12.0,10.0
4,14.0,14.0,16.0,16.0,8.0


In [58]:
clasificador_NB = CategoricalNB(alpha=1)  # alpha es el parámetro de suavizado
clasificador_NB.fit(atributos_discretizados, objetivo)
predicciones = clasificador_NB.predict(atributos_discretizados)
confusion_matrix(objetivo, predicciones)
recall_score(objetivo, predicciones, average='micro')

0.45549621717846017

In [59]:
print("Parámetros usados: ")
clasificador_NB.get_params()

Parámetros usados: 


{'alpha': 1,
 'class_prior': None,
 'fit_prior': True,
 'force_alpha': True,
 'min_categories': None}

## <a name="knn"></a> **KNN**

In [60]:
normalizador = MinMaxScaler(
    # Cada atributo se normaliza al intervalo [0, 1]
    feature_range=(0, 1)
)

# Como nos interesa conservar los atributos originales, realizamos la
# normalización sobre una copia del DataFrame de atributos
atributos_normalizados = atributos.copy()
atributos_normalizados[:] = normalizador.fit_transform(atributos_normalizados)
atributos_normalizados.head()

,degree_centrality,k_path_centrality,closeness_centrality,harmonic_centrality,clustering
0,0.000000,0.000000,0.374144,0.342296,0.000000
1,0.046610,0.046610,0.706650,0.684463,0.481283
2,0.015537,0.015537,0.405322,0.375567,0.651515
3,0.012712,0.012712,0.535493,0.499703,0.511111
4,0.070621,0.070621,0.585767,0.558742,0.420392


In [61]:
clasificador_kNN = KNeighborsClassifier(
    # Para cada ejemplo se consideran los 5 ejemplos más cercanos
    n_neighbors=2,
    # La cercanía viene determinada por la distancia euclídea
    metric='euclidean'
)
clasificador_kNN.fit(atributos_normalizados, objetivo)
predicciones = clasificador_kNN.predict(atributos_normalizados)
confusion_matrix(objetivo, predicciones)
recall_score(objetivo, predicciones, average = 'micro')

0.7582554517133956

In [62]:
print("Parámetros usados: ")
clasificador_kNN.get_params()

Parámetros usados: 


{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'euclidean',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 2,
 'p': 2,
 'weights': 'uniform'}

## <a name="random-forest"></a> **Random forest**

In [63]:
# Tutorial: https://www.datacamp.com/tutorial/random-forests-classifier-python
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
clasificador_RF = RandomForestClassifier(n_estimators = 200, max_depth=None)
clasificador_RF.fit(atributos, objetivo)
predicciones = clasificador_RF.predict(atributos)
confusion_matrix(objetivo, predicciones)
recall_score(objetivo, predicciones, average = 'micro')

0.9986203827325323

In [64]:
print("Parámetros usados: ")
clasificador_RF.get_params()

Parámetros usados: 


{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 200,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

## <a name="gradient-boosting"></a> **Gradient Boosting**

In [65]:
clasificador_GB = GradientBoostingClassifier(random_state=42)
clasificador_GB.fit(atributos, objetivo)
predicciones = clasificador_GB.predict(atributos)
confusion_matrix(objetivo, predicciones)
recall_score(objetivo, predicciones, average = 'micro')

0.5773920783266577

In [66]:
print("Parámetros usados: ")
clasificador_GB.get_params()

Parámetros usados: 


{'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'log_loss',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'random_state': 42,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}